In [ ]:
import pandas as pd
import re
import numpy as np

Data preprocessing, converting to numpy array and dividing into train, test, validation dataset. Data location of both glove matrix, sentiment(2), movie.txt and data3.csv(CBOW matrix have to be changed) has to be changed.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pat='/content/drive/My Drive/Projects/STCS assignment(Word vectors)/'

In [ ]:
#sentiment score
df = pd.read_csv (pat+'sentiment (2).csv')
print (df)

In [ ]:
reviews = df['review']
labels = np.array(df['senti_score'])

In [ ]:
#assigning label
for label in range(len(labels)):
  if float(labels[label])<=0.2:
    labels[label] = 0
  elif float(labels[label])<=0.4:
    labels[label] = 1
  elif float(labels[label])<=0.6:
    labels[label] = 2
  elif float(labels[label])<=0.8:
    labels[label] = 3
  elif float(labels[label])<=1.0:
    labels[label] = 4

In [ ]:
reviews = reviews.str.lower()
for i in range(df.shape[0]):
  reviews[i] = re.sub(r'[^\w\s]', '', reviews[i])

In [ ]:
a=np.loadtxt(pat+'Movie Review Dataset.txt',delimiter='\t',dtype=str)

In [ ]:
a2=[]
for e1,e2 in a:
    a2+=[e2]

In [ ]:
len(a2)

11856

In [ ]:
#generating vocabulary
q2=[] #contains word of sentences
q3=[] #contains sentences
for e in a2:
  f=[]
  for el2 in e.split(' '):
    f+=[el2.lower()]
    
  q2+=f
  q3+=[f]
sen_len=np.array(q2) 
n=len(np.unique(sen_len))#Vocabulary size
n

19538

In [ ]:
qwe=np.unique(sen_len)#list of unique words
tab=np.zeros([n,n],dtype=np.int32)#coocurrence matrix
word_to_num=dict(zip(qwe,range(n)))#dictionary for word to num
sen=np.array(q3)
data_training = sen[1:]#remove heading

In [ ]:
#loading CBOW vector
wemb=np.loadtxt(pat+'data3.csv',delimiter=',')
wemb.T.shape

(19538, 400)

In [ ]:
len(qwe)

19538

In [ ]:
#creating a word to vector dictionary
embeddings_dictionary=dict(zip(qwe,wemb.T))

In [ ]:
maxlen = 32

In [ ]:
#creating converting entences to embeddings
embedding_matrix = np.empty((len(reviews),maxlen,100))
j=0
k=0
k2=0
for sent in reviews:
  sent_embed = np.empty((maxlen,100))
  i=0
  for word in sent.split():
    if i==maxlen:
      break
    
    try:
      sent_embed[i] = np.array(embeddings_dictionary[word]).reshape((1,100))
    except KeyError: 
      sent_embed[i] = np.zeros((1,100))
      k+=1
    print(sent_embed.shape)
    i+=1
    k2+=1

  if i<maxlen:
    padding_matrix = np.zeros((maxlen-i,100))
    sent_embed[i:] = padding_matrix
  embedding_matrix[j] = sent_embed
  j+=1

In [ ]:
X = embedding_matrix

In [ ]:
#assigning label to y
y = np.empty((len(reviews),maxlen))
com = np.ones((1,maxlen))
i=0
for label in labels:
  y[i] = np.array(label*com)
  i+=1
y = y.astype(int)

In [ ]:
#Taking only first index(code from RNN)
Y=y[:,0]

In [ ]:
#loading data split
sent=np.loadtxt(pat+'Train-Dev_test split.txt',delimiter=',',dtype=str,skiprows=1)

In [ ]:
sent

array([['1', '1'],
       ['2', '1'],
       ['3', '2'],
       ...,
       ['11853', '1'],
       ['11854', '1'],
       ['11855', '1']], dtype='<U5')

In [ ]:
#splitting data
X_train = np.empty((7980,maxlen,100))
X_test = np.empty((1101,maxlen,100)) 
X_val = np.empty((2205,maxlen,100))
y_train = np.empty((7980))
y_test = np.empty((1101))
y_val = np.empty((2205))
i=0
j=0
k=0

In [ ]:
#splitting data
b=0
for k in range(len(reviews)):
  if sent[k][1]=='1':
    X_train[i] = X[min(11285,int(sent[k][0])-1)]
    y_train[i] = Y[int(sent[k][0])-1]
    i+=1
    
  elif sent[k][1]=='2':
    X_val[b] = X[min(11285,int(sent[k][0])-1)]
    y_val[b] = Y[int(sent[k][0])-1]
    
    b+=1
  elif sent[k][1]=='3':
    X_test[j] = X[min(11285,int(sent[k][0])-1)]
    y_test[j] = Y[int(sent[k][0])-1]
    j+=1
    
y_train = y_train.astype(int)
y_val = y_val.astype(int)
y_test = y_test.astype(int)
i,b,j

(7980, 2205, 1101)

In [ ]:
X_train[0]

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.57058001,  0.44183001,  0.70102   , ..., -0.66101998,
         0.47196999,  0.37253001],
       [ 0.41051999,  0.058649  ,  0.28514999, ..., -0.37898001,
         0.33021   , -0.27462   ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten,LSTM
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

data preparation IMP ---- change the location of sentiment(2) and glove matrix while running

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
movie_reviews = pd.read_csv(pat+'sentiment (2).csv')
movie_reviews.isnull().values.any()
movie_reviews.shape

(11286, 3)

text preprocessing - removing stop-words, punctuation

In [ ]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [ ]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

Tokenizing the words and padding them if necessary

In [ ]:
#loading glove vectors
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open(pat+'glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [ ]:
#embedding matrixfor embedding layer
embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

Model, and the hyperparameter testing code.

In [ ]:
converting onehot to normal numbers
X_tr=np.zeros(X_train.shape[:2])
i=0
for el in X_train:
  j=0
  for e in el:
    X_tr[i,j]=np.argmax(e)
    j+=1
  i+=1
X_t=np.zeros(X_test.shape[:2])
i=0
for el in X_test:
  j=0
  for e in el:
    X_t[i,j]=np.argmax(e)
    j+=1
  i+=1
X_v=np.zeros(X_val.shape[:2])
i=0
for el in X_val:
  j=0
  for e in el:
    X_v[i,j]=np.argmax(e)
    j+=1
  i+=1

In [ ]:
#one- hot function
def oh(y):
  y2=np.zeros([len(y),5])
  i=0
  for el in y:
    y2[i,el]=1
    i+=1
  return y2

In [ ]:
oh([1,2,3,4])

array([[0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

The model

In [ ]:
model2 = Sequential()
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model2.add(embedding_layer)
model2.add(LSTM(128))

model2.add(Dense(5, activation='sigmoid'))
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
print(model2.summary())
history = model2.fit(X_tr, oh(y_train[:]), batch_size=128, epochs=10, verbose=1, validation_data=[X_v,oh(y_val[:])])
score = model2.evaluate(X_t, oh(y_test[:]), verbose=1)

Testing for hyperparameter

In [ ]:
num = [64,128,256,200]     #model running by testing various hyperparameter
losses = []
for i in num:
  model = Sequential()
  embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)
  model.add(embedding_layer)
  model.add(LSTM(i))

  model.add(Dense(5, activation='sigmoid'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
  print(model.summary())
  history = model.fit(X_tr, oh(y_train[:]), batch_size=128, epochs=10, verbose=1, validation_data=[X_v,oh(y_val[:])])
  score = model.evaluate(X_t, oh(y_test[:]), verbose=1)
  losses.append(score)

In [ ]:
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

Printing the values of train and test accuracy

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

Code testing, do not run

In [ ]:
reviews = df['review']
labels = np.array(df['senti_score'])

In [ ]:
for label in range(len(labels)):
  if float(labels[label])<=0.2:
    labels[label] = 0
  elif float(labels[label])<=0.4:
    labels[label] = 1
  elif float(labels[label])<=0.6:
    labels[label] = 2
  elif float(labels[label])<=0.8:
    labels[label] = 3
  elif float(labels[label])<=1.0:
    labels[label] = 4

In [ ]:
reviews = reviews.str.lower()
for i in range(df.shape[0]):
  reviews[i] = re.sub(r'[^\w\s]', '', reviews[i])

Everything is same, for CBOW

In [ ]:
a=np.loadtxt(pat+'Movie Review Dataset.txt',delimiter='\t',dtype=str)

In [ ]:
a2=[]
for e1,e2 in a:
    a2+=[e2]

In [ ]:
len(a2)

11856

In [ ]:
q2=[] #contains word of sentences
q3=[] #contains sentences
for e in a2:
  f=[]
  for el2 in e.split(' '):
    f+=[el2.lower()]
    
  q2+=f
  q3+=[f]
sen_len=np.array(q2) 
n=len(np.unique(sen_len))#Vocabulary size
n

19538

In [ ]:
qwe=np.unique(sen_len)#list of unique words
tab=np.zeros([n,n],dtype=np.int32)#coocurrence matrix
word_to_num=dict(zip(qwe,range(n)))#dictionary for word to num
sen=np.array(q3)
data_training = sen[1:]#remove heading

In [ ]:
wemb=np.loadtxt(pat+'data3.csv',delimiter=',')
wemb.T.shape

(19538, 400)

In [ ]:
len(qwe)

19538

In [ ]:
embeddings_dictionary=dict(zip(qwe,wemb.T))

In [ ]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
#glove_file = open('/content/drive/MyDrive/glove.6B.100d.txt', encoding="utf8")
wemb=np.loadtxt(pat+'data3.csv',delimiter=',')



In [ ]:
maxlen = 32

In [ ]:
embedding_matrix = np.empty((len(reviews),maxlen,400))
j=0
k=0
k2=0
for sent in reviews:
  sent_embed = np.empty((maxlen,400))
  i=0
  for word in sent.split():
    if i==maxlen:
      break
    
    try:
      sent_embed[i] = np.array(embeddings_dictionary[word]).reshape((1,400))
    except KeyError: 
      sent_embed[i] = np.zeros((1,400))
      k+=1
    print(sent_embed.shape)
    i+=1
    k2+=1

  if i<maxlen:
    padding_matrix = np.zeros((maxlen-i,400))
    sent_embed[i:] = padding_matrix
  embedding_matrix[j] = sent_embed
  j+=1

In [ ]:
X = embedding_matrix

In [ ]:
y = np.empty((len(reviews),maxlen))
com = np.ones((1,maxlen))
i=0
for label in labels:
  y[i] = np.array(label*com)
  i+=1
y = y.astype(int)

In [ ]:
Y=y[:,0]

In [ ]:
# fp = open('/content/Train-Dev_test split.txt', 'r')
sent=np.loadtxt(pat+'Train-Dev_test split.txt',delimiter=',',dtype=str,skiprows=1)

In [ ]:
sent

array([['1', '1'],
       ['2', '1'],
       ['3', '2'],
       ...,
       ['11853', '1'],
       ['11854', '1'],
       ['11855', '1']], dtype='<U5')

In [ ]:
X_train = np.empty((7980,maxlen,400))
X_test = np.empty((1101,maxlen,400)) 
X_val = np.empty((2205,maxlen,400))
y_train = np.empty((7980))
y_test = np.empty((1101))
y_val = np.empty((2205))
i=0
j=0
k=0

In [ ]:
print(labels)

[3. 4. 2. ... 3. 0. 1.]


In [ ]:
a=0
b=0
c=0
for k in range(len(reviews)):
  if sent[k][1]=='1':
    X_train[i] = X[min(11285,int(sent[k][0])-1)]
    y_train[i] = Y[int(sent[k][0])-1]
    i+=1
    a+=1
  elif sent[k][1]=='2':
    X_val[b] = X[min(11285,int(sent[k][0])-1)]
    y_val[b] = Y[int(sent[k][0])-1]
    
    b+=1
  elif sent[k][1]=='3':
    X_test[j] = X[min(11285,int(sent[k][0])-1)]
    y_test[j] = Y[int(sent[k][0])-1]
    j+=1
    
y_train = y_train.astype(int)
y_val = y_val.astype(int)
y_test = y_test.astype(int)
i,b,j

(7980, 2205, 1101)

In [ ]:
X_train[0]

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.57058001,  0.44183001,  0.70102   , ..., -0.66101998,
         0.47196999,  0.37253001],
       [ 0.41051999,  0.058649  ,  0.28514999, ..., -0.37898001,
         0.33021   , -0.27462   ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

data preparation IMP ---- change the location of sentiment(2) and glove matrix while running

text preprocessing - removing stop-words, punctuation

Tokenizing the words and padding them if necessary

Model, and the hyperparameter testing code.

In [ ]:
X_tr=np.zeros(X_train.shape[:2])
i=0
for el in X_train:
  j=0
  for e in el:
    X_tr[i,j]=np.argmax(e)
    j+=1
  i+=1
X_t=np.zeros(X_test.shape[:2])
i=0
for el in X_test:
  j=0
  for e in el:
    X_t[i,j]=np.argmax(e)
    j+=1
  i+=1
X_v=np.zeros(X_val.shape[:2])
i=0
for el in X_val:
  j=0
  for e in el:
    X_v[i,j]=np.argmax(e)
    j+=1
  i+=1

In [ ]:
X_train[1]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
X_t=np.ndarray(X_t,shape=[1101,32])
X_tr=np.array(X_tr)
X_v=np.array(X_v)

TypeError: ignored

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
err=model2.predict(X_t)

In [ ]:
def oh(y):
  y2=np.zeros([len(y),5])
  i=0
  for el in y:
    y2[i,el]=1
    i+=1
  return y2

In [ ]:
oh([1,2,3,4])

array([[0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [ ]:
import tensorflow as tf

In [ ]:
model2 = Sequential()
embedding_layer = Embedding(vocab_size, 400, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model2.add(embedding_layer)
model2.add(LSTM(128))

model2.add(Dense(5, activation='softmax'))
model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['acc'])
print(model2.summary())
history = model2.fit(X_tr, oh(y_train[:]), batch_size=128, epochs=100, verbose=1, shuffle=True,validation_data=[X_v,oh(y_val[:])])
score = model2.evaluate(X_t, oh(y_test[:]), verbose=1)

ValueError: ignored

In [ ]:
model2.predict(X_tr)

array([[0.13175671, 0.25283512, 0.20003451, 0.263257  , 0.15211672],
       [0.13175671, 0.25283512, 0.20003451, 0.263257  , 0.15211672],
       [0.13175671, 0.25283512, 0.20003451, 0.263257  , 0.15211672],
       ...,
       [0.13175671, 0.25283512, 0.20003451, 0.263257  , 0.15211672],
       [0.13175671, 0.25283512, 0.20003451, 0.263257  , 0.15211672],
       [0.13175671, 0.25283512, 0.20003451, 0.263257  , 0.1521167 ]],
      dtype=float32)